In [9]:

import pandas as pd
import duckdb 
import os
from google import genai
from google.genai import types
import google.generativeai as genai

In [1]:
import pandas as pd

In [2]:

chicago_crime = pd.read_csv("./data/chicago_crime_2023_2025.csv")
chicago_crime["updated_on"] = pd.to_datetime(chicago_crime["updated_on"])
chicago_crime["x_coordinate"] = chicago_crime["x_coordinate"].astype(float)
chicago_crime["y_coordinate"] = chicago_crime["y_coordinate"].astype(float)
chicago_crime["latitude"] = chicago_crime["latitude"].astype(float)
chicago_crime["longitude"] = chicago_crime["longitude"].astype(float)
chicago_crime["year"]= chicago_crime["year"].astype(int)

In [3]:
chicago_crime.head()

,id,case_number,datetime,block,iucr,primary_type,description,location_description,arrest,domestic,...,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude
0,13140855,JG341458,2023-01-01 00:00:00,082XX S JEFFERY BLVD,1754,OFFENSE INVOLVING CHILDREN,AGGRAVATED SEXUAL ASSAULT OF CHILD BY FAMILY M...,APARTMENT,False,True,...,4,8.0,46.0,02,1190953.0,1850848.0,2023,2023-09-24 15:41:26,41.745739,-87.575883
1,13180096,JG387858,2023-01-01 00:00:00,075XX S WOLCOTT AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,6,17.0,71.0,11,1164996.0,1854651.0,2023,2023-08-20 15:40:56,41.756762,-87.670887
2,13168471,JG374193,2023-01-01 00:00:00,013XX W HARRISON ST,0460,BATTERY,SIMPLE,SCHOOL - PUBLIC GROUNDS,False,False,...,12,34.0,28.0,08B,1167465.0,1897475.0,2023,2023-08-19 15:40:26,41.874223,-87.660610
3,13078152,JG267031,2023-01-01 00:00:00,101XX S BEVERLY AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,22,21.0,73.0,11,1168800.0,1837525.0,2023,2023-08-19 15:40:26,41.709685,-87.657439
4,13120699,JG314178,2023-01-01 00:00:00,063XX N FAIRFIELD AVE,1544,SEX OFFENSE,SEXUAL EXPLOITATION OF A CHILD,OTHER (SPECIFY),False,False,...,24,50.0,2.0,17,1156847.0,1941985.0,2023,2023-08-19 15:40:26,41.996584,-87.698384


In [4]:

print(chicago_crime.dtypes)

id                               int64
case_number                     object
datetime                        object
block                           object
iucr                            object
primary_type                    object
description                     object
location_description            object
arrest                            bool
domestic                          bool
beat                             int64
district                         int64
ward                           float64
community_area                 float64
fbi_code                        object
x_coordinate                   float64
y_coordinate                   float64
year                             int64
updated_on              datetime64[ns]
latitude                       float64
longitude                      float64
dtype: object


In [ ]:
from google import genai

client = genai.Client(api_key=apikey)

In [16]:
prompt1 = """
        I am working with a dataset that contains information about Chicago crime incidents. 
        I want to create a SQL query to pull the total number of crimes that ended in arrest by a year
        """

In [17]:
response1 = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt1,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0
    )
)

In [20]:
print(response1.candidates[0].content.parts[0].text)

```sql
SELECT
    STRFTIME('%Y', Date) AS CrimeYear,  -- Extract the year from the 'Date' column
    COUNT(*) AS TotalArrests  -- Count the number of crimes for each year
FROM
    ChicagoCrimes  -- Replace 'ChicagoCrimes' with the actual name of your table
WHERE
    Arrest = TRUE  -- Filter for crimes where an arrest was made
GROUP BY
    CrimeYear  -- Group the results by year
ORDER BY
    CrimeYear;  -- Order the results by year (optional, for readability)
```

**Explanation:**

1. **`SELECT STRFTIME('%Y', Date) AS CrimeYear`**:
   - `STRFTIME('%Y', Date)`: This is the key part for extracting the year.  `STRFTIME` is a SQLite function (and similar functions exist in other SQL dialects like MySQL, PostgreSQL, etc.) that formats a date/time value.  `'%Y'` specifies that we want to extract the year in a four-digit format (e.g., 2023).  `Date` is assumed to be the name of the column in your table that stores the date and time of the crime incident.  **Important:**  Make sure the `Date` c

In [21]:
prompt2 = """I am working with a dataset that contains information about Chicago crime incidents. 
        The table name is 'chicago_crime'. I want to create a SQL query to pull the total number of crimes that ended in arrest by a year. 
        The columns are: id, case_number, date, block, primary_type, description, location_description, arrest, domestic, beat, district, ward, community_area, fbi_code, x_coordinate, y_coordinate, year, updated_on. Please provide the SQL query.
        """

In [22]:

response2 = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt2,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0
    )
)

In [23]:
print(response2.candidates[0].content.parts[0].text)

```sql
SELECT
    year,
    COUNT(*) AS total_arrests
FROM
    chicago_crime
WHERE
    arrest = TRUE  -- Assuming 'arrest' is a boolean column (TRUE/FALSE)
    -- If 'arrest' is a string column ('TRUE'/'FALSE' or 'Yes'/'No'), adjust the WHERE clause accordingly.
    -- For example: WHERE arrest = 'TRUE' or WHERE arrest = 'Yes'
GROUP BY
    year
ORDER BY
    year;
```

**Explanation:**

1. **`SELECT year, COUNT(*) AS total_arrests`**:  This selects the `year` and the count of all rows for each year, aliasing the count as `total_arrests`.
2. **`FROM chicago_crime`**:  This specifies the table to retrieve data from.
3. **`WHERE arrest = TRUE`**: This filters the data to include only rows where the `arrest` column indicates an arrest was made.  **Important:**  This assumes your `arrest` column is a boolean (TRUE/FALSE).  If it's a string (e.g., 'TRUE', 'FALSE', 'Yes', 'No'), you'll need to adjust this condition accordingly (see the comment in the code).
4. **`GROUP BY year`**: This groups 

In [24]:
sql = """
SELECT
    year,
    COUNT(*) AS total_arrests
FROM
    chicago_crime
WHERE
    arrest = TRUE  -- Assuming 'arrest' is a boolean column (TRUE/FALSE)
    -- If 'arrest' is a string column ('TRUE'/'FALSE' or 'Yes'/'No'), adjust the WHERE clause accordingly.
    -- For example: WHERE arrest = 'TRUE' or WHERE arrest = 'Yes'
GROUP BY
    year
ORDER BY
    year;
    """

In [25]:

duckdb.sql(sql).show()

┌───────┬───────────────┐
│ year  │ total_arrests │
│ int64 │     int64     │
├───────┼───────────────┤
│  2023 │         31998 │
│  2024 │         35393 │
│  2025 │         12609 │
└───────┴───────────────┘



In [26]:
prompt_template = """I am working with a dataset that contains information about Chicago crime incidents. 
                    The table name is 'chicago_crime'. I want to create a SQL query that answers the following question: "{}". 
                    The columns are: id, case_number, date, block, primary_type, description, location_description, arrest, domestic, beat, district, ward, community_area, fbi_code, x_coordinate, y_coordinate, year, updated_on. Please provide the SQL query.
                """

In [27]:

user_question = "How many cases ended up with arrest during 2024?"
prompt3 = prompt_template.format(user_question)

In [28]:
response3 = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt3,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0
    )
)

In [29]:
print(response3.candidates[0].content.parts[0].text)

```sql
SELECT COUNT(*) AS arrest_count
FROM chicago_crime
WHERE year = 2024
  AND arrest = TRUE;
```

**Explanation:**

*   **`SELECT COUNT(*) AS arrest_count`**: This counts all rows that meet the specified criteria and names the resulting count `arrest_count`.
*   **`FROM chicago_crime`**: This specifies that we are querying the `chicago_crime` table.
*   **`WHERE year = 2024`**: This filters the data to include only incidents that occurred in the year 2024.
*   **`AND arrest = TRUE`**: This further filters the data to include only incidents where an arrest was made.  The `arrest` column is assumed to be a boolean (TRUE/FALSE) indicating whether an arrest occurred.  If the `arrest` column is stored as a different data type (e.g., 'Y'/'N', 1/0), you'll need to adjust this condition accordingly (e.g., `AND arrest = 'Y'` or `AND arrest = 1`).



In [30]:
sql = """
SELECT COUNT(*) AS arrest_count
FROM chicago_crime
WHERE year = 2024
  AND arrest = TRUE;
"""

In [31]:
duckdb.sql(sql).show()

┌──────────────┐
│ arrest_count │
│    int64     │
├──────────────┤
│        35393 │
└──────────────┘



In [32]:
system_template = """
Given the following SQL table, your job is to write queries given a user’s request. \n
CREATE TABLE {} ({}) \n
"""

In [33]:
user_template = "Write a SQL query that returns - {}"

In [34]:
table_name = "chicago_crime" 
tbl_description = duckdb.sql("DESCRIBE SELECT * FROM " + table_name +  ";")
tbl_description

┌──────────────────────┬──────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name      │ column_type  │  null   │   key   │ default │  extra  │
│       varchar        │   varchar    │ varchar │ varchar │ varchar │ varchar │
├──────────────────────┼──────────────┼─────────┼─────────┼─────────┼─────────┤
│ id                   │ BIGINT       │ YES     │ NULL    │ NULL    │ NULL    │
│ case_number          │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ datetime             │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ block                │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ iucr                 │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ primary_type         │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ description          │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ location_description │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ arrest               │ BOOLEAN      │ 

In [35]:
col_attr = tbl_description.df()[["column_name", "column_type"]]
col_attr["column_joint"] = col_attr["column_name"] + " " +  col_attr["column_type"]
col_names = str(list(col_attr["column_joint"].values)).replace('[', '').replace(']', '').replace('\'', '')
col_names

'id BIGINT, case_number VARCHAR, datetime VARCHAR, block VARCHAR, iucr VARCHAR, primary_type VARCHAR, description VARCHAR, location_description VARCHAR, arrest BOOLEAN, domestic BOOLEAN, beat BIGINT, district BIGINT, ward DOUBLE, community_area DOUBLE, fbi_code VARCHAR, x_coordinate DOUBLE, y_coordinate DOUBLE, year BIGINT, updated_on TIMESTAMP_NS, latitude DOUBLE, longitude DOUBLE'

In [36]:

system = system_template.format(table_name, col_names)
print(system)


Given the following SQL table, your job is to write queries given a user’s request. 

CREATE TABLE chicago_crime (id BIGINT, case_number VARCHAR, datetime VARCHAR, block VARCHAR, iucr VARCHAR, primary_type VARCHAR, description VARCHAR, location_description VARCHAR, arrest BOOLEAN, domestic BOOLEAN, beat BIGINT, district BIGINT, ward DOUBLE, community_area DOUBLE, fbi_code VARCHAR, x_coordinate DOUBLE, y_coordinate DOUBLE, year BIGINT, updated_on TIMESTAMP_NS, latitude DOUBLE, longitude DOUBLE) 




In [37]:
question = "How many cases ended up with arrest during 2024?"
user = user_template.format(question)
print(user)

Write a SQL query that returns - How many cases ended up with arrest during 2024?


In [38]:
prompt4 = [system, user]

In [39]:

response4 = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt4,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0
    )
)

In [40]:

print(response4.candidates[0].content.parts[0].text)

```sql
SELECT
  COUNT(CASE WHEN arrest = TRUE THEN 1 ELSE NULL END)
FROM chicago_crime
WHERE
  year = 2024;
```


In [41]:

import re

def is_markdown_code_chunk(text):
    """
    Checks if the given text is in Markdown code chunk format.

    Args:
        text (str): The text to check.

    Returns:
        bool: True if the text is in Markdown code chunk format, False otherwise.
    """
    pattern = r"```[^`]*```"
    return bool(re.search(pattern, text, re.DOTALL))

def extract_code_from_markdown(markdown_text):
    """
    Extracts code from a Markdown code chunk.

    Args:
        markdown_text (str): The Markdown text containing the code chunk.

    Returns:
        str: The extracted code.
    """
    pattern = r"```(.*?)\n(?P<code>.*?)\n```"
    match = re.search(pattern, markdown_text, re.DOTALL)
    if match:
        return match.group("code")
    else:
        return None

In [42]:
sql_chunk  = response4.candidates[0].content.parts[0].text

if is_markdown_code_chunk(text= sql_chunk):
    sql = extract_code_from_markdown(markdown_text= sql_chunk)
else:
    sql = sql_chunk

print(sql_chunk)
print(sql)

```sql
SELECT
  COUNT(CASE WHEN arrest = TRUE THEN 1 ELSE NULL END)
FROM chicago_crime
WHERE
  year = 2024;
```
SELECT
  COUNT(CASE WHEN arrest = TRUE THEN 1 ELSE NULL END)
FROM chicago_crime
WHERE
  year = 2024;


In [43]:

duckdb.sql(sql).show()

┌────────────────────────────────────────────────────────────────────────────┐
│ count(CASE  WHEN ((arrest = CAST('t' AS BOOLEAN))) THEN (1) ELSE NULL END) │
│                                   int64                                    │
├────────────────────────────────────────────────────────────────────────────┤
│                                                                      35393 │
└────────────────────────────────────────────────────────────────────────────┘



In [45]:

def create_message(table_name, query):

    class message:
        def __init__(message, system, user, column_names, column_attr):
            message.system = system
            message.user = user
            message.column_names = column_names
            message.column_attr = column_attr

    
    system_template = """

    Given the following SQL table, your job is to write queries given a user’s request. Return just the SQL query as plan text, without additional text and don't use markdown format. \n

    CREATE TABLE {} ({}) \n
    """

    user_template = "Write a SQL query that returns - {}"
    
    tbl_describe = duckdb.sql("DESCRIBE SELECT * FROM " + table_name +  ";")
    col_attr = tbl_describe.df()[["column_name", "column_type"]]
    col_attr["column_joint"] = col_attr["column_name"] + " " +  col_attr["column_type"]
    col_names = str(list(col_attr["column_joint"].values)).replace('[', '').replace(']', '').replace('\'', '')

    system = system_template.format(table_name, col_names)
    user = user_template.format(query)

    m = message(system = system, user = user, column_names = col_attr["column_name"], column_attr = col_attr["column_type"])
    return m

functionalizw the process


In [46]:

query = "How many cases ended up with arrest?"
msg = create_message(table_name = "chicago_crime", query = query)

In [53]:
print(msg.system)



    Given the following SQL table, your job is to write queries given a user’s request. Return just the SQL query as plan text, without additional text and don't use markdown format. 


    CREATE TABLE chicago_crime (id BIGINT, case_number VARCHAR, datetime VARCHAR, block VARCHAR, iucr VARCHAR, primary_type VARCHAR, description VARCHAR, location_description VARCHAR, arrest BOOLEAN, domestic BOOLEAN, beat BIGINT, district BIGINT, ward DOUBLE, community_area DOUBLE, fbi_code VARCHAR, x_coordinate DOUBLE, y_coordinate DOUBLE, year BIGINT, updated_on TIMESTAMP_NS, latitude DOUBLE, longitude DOUBLE) 

    


In [48]:

prompt = [msg.system, msg.user]

In [49]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents= prompt,
    config=types.GenerateContentConfig(
        max_output_tokens=500,
        temperature=0,
        top_p = 1,
        top_k = 32
    )
)

In [51]:

code = response.candidates[0].content.parts[0].text
print(code)
code = extract_code_from_markdown(markdown_text=code)
print(code)

```sql
SELECT count(*) FROM chicago_crime WHERE arrest = TRUE
```
SELECT count(*) FROM chicago_crime WHERE arrest = TRUE


In [ ]:

def lang2sql_gemini( table_name, query, model ="gemini-2.0-flash", temperature = 0,max_output_token = 256):
    class lang2sql:
        def __init__(output, message, response, sql):
            output.message = message
            output.response = response
            output.sql = sql      

    client = genai.Client(api_key = api_key)

    m = create_message(table_name = table_name, query = query)

    prompt = [m.system, m.user]

    response = client.models.generate_content(
    model=model,
    contents= prompt,
    config=types.GenerateContentConfig(
        max_output_tokens=max_output_token,
        temperature=temperature
        )
    )

    sql = response.candidates[0].content.parts[0].text
    if is_markdown_code_chunk(text = sql):
        sql = extract_code_from_markdown(markdown_text=sql)

    output = lang2sql(message = m, response = response, sql = sql)
    return output

In [55]:
query = lang2sql_gemini( 
                        table_name = "chicago_crime", 
                        query = "How many cases ended up with arrest?")

In [56]:
print(query.sql)
duckdb.sql(query.sql).show()

SELECT count(*) FROM chicago_crime WHERE arrest = TRUE
┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        80000 │
└──────────────┘



In [62]:
def lang2answer_gemini( table_name, query, model ="gemini-2.0-flash", temperature = 0,max_output_token = 256, verbose = False):
    query = lang2sql_gemini(
                        table_name = table_name,
                        query = query, 
                        model = model,
                        temperature = temperature,
                        max_output_token = max_output_token
                    )
    if verbose:
        print(query.sql)
    output = duckdb.sql(query.sql)
    return output

In [63]:
lang2answer_gemini( 
            table_name = "chicago_crime", 
            query = "How many cases ended up with arrest?",
            verbose = True)

SELECT count(*) FROM chicago_crime WHERE arrest = TRUE


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        80000 │
└──────────────┘

In [65]:
lang2answer_gemini( 
            table_name = "chicago_crime", 
            query = "Summarize the cases by primary type",
            verbose = True)

SELECT primary_type, COUNT(*) AS case_count FROM chicago_crime GROUP BY primary_type;


┌───────────────────────────────────┬────────────┐
│           primary_type            │ case_count │
│              varchar              │   int64    │
├───────────────────────────────────┼────────────┤
│ OTHER OFFENSE                     │      38440 │
│ CRIMINAL DAMAGE                   │      66226 │
│ DECEPTIVE PRACTICE                │      37780 │
│ ASSAULT                           │      52628 │
│ SEX OFFENSE                       │       2999 │
│ BURGLARY                          │      18328 │
│ NARCOTICS                         │      14297 │
│ CONCEALED CARRY LICENSE VIOLATION │        510 │
│ INTIMIDATION                      │        438 │
│ LIQUOR LAW VIOLATION              │        453 │
│   ·                               │         ·  │
│   ·                               │         ·  │
│   ·                               │         ·  │
│ ARSON                             │       1123 │
│ KIDNAPPING                        │        269 │
│ GAMBLING                     

In [66]:

lang2answer_gemini( 
            table_name = "chicago_crime", 
            query = "Summarize the cases by primary type, sort by number of crimes in descending order",
            verbose = True)

SELECT primary_type, count(*) AS num_crimes FROM chicago_crime GROUP BY primary_type ORDER BY num_crimes DESC


┌───────────────────────────────────┬────────────┐
│           primary_type            │ num_crimes │
│              varchar              │   int64    │
├───────────────────────────────────┼────────────┤
│ THEFT                             │     134864 │
│ BATTERY                           │     103032 │
│ CRIMINAL DAMAGE                   │      66226 │
│ MOTOR VEHICLE THEFT               │      55948 │
│ ASSAULT                           │      52628 │
│ OTHER OFFENSE                     │      38440 │
│ DECEPTIVE PRACTICE                │      37780 │
│ ROBBERY                           │      21972 │
│ BURGLARY                          │      18328 │
│ WEAPONS VIOLATION                 │      18250 │
│         ·                         │         ·  │
│         ·                         │         ·  │
│         ·                         │         ·  │
│ CONCEALED CARRY LICENSE VIOLATION │        510 │
│ LIQUOR LAW VIOLATION              │        453 │
│ INTIMIDATION                 

In [67]:
lang2answer_gemini( 
            table_name = "chicago_crime", 
            query = "Show me cases that are type of robbery", 
            verbose=True)

SELECT * FROM chicago_crime WHERE primary_type = 'ROBBERY'


┌──────────┬─────────────┬─────────────────────┬────────────────────────┬─────────┬──────────────┬────────────────────────────────────┬──────────────────────┬─────────┬──────────┬───────┬──────────┬────────┬────────────────┬──────────┬──────────────┬──────────────┬───────┬─────────────────────┬──────────────┬───────────────┐
│    id    │ case_number │      datetime       │         block          │  iucr   │ primary_type │            description             │ location_description │ arrest  │ domestic │ beat  │ district │  ward  │ community_area │ fbi_code │ x_coordinate │ y_coordinate │ year  │     updated_on      │   latitude   │   longitude   │
│  int64   │   varchar   │       varchar       │        varchar         │ varchar │   varchar    │              varchar               │       varchar        │ boolean │ boolean  │ int64 │  int64   │ double │     double     │ varchar  │    double    │    double    │ int64 │    timestamp_ns     │    double    │    double     │
├──────────┼───────

In [70]:
lang2answer_gemini( 
                        table_name = "chicago_crime", 
                        query = "How many cases is the type of ROBBERY?",
                        verbose = True)

SELECT count(*) FROM chicago_crime WHERE primary_type = 'ROBBERY'


┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        21972 │
└──────────────┘